## Домашнее задание №4

В качестве домашнего задания предлагаются следующие задачи:

1) Реализовать метод аугментации текстовых данных на основе дистрибутивных векторных представлений, который можно применить в перечисленных ниже задачах (на выбор) и проанализировать итоговое качество с/без аугментации.

2) Решить задачу регрессии с использованием текстовых признаков на англоязычном датасете [Medium Stories](https://www.kaggle.com/harrisonjansma/medium-stories).

3) Решить задачу детекции парафразов.

4) Использовать методы для нахождения оптимальных гиперпараметров модели.

5) Использовать кросс-валидацию и/или методы семплирования данных.

Это домашнее задание является "конструктором": вы можете выполнить интересующие вас блоки или получить больше 10 баллов.

Если качество моделей не повысилось, но попытка реализована верно, вы все равно получаете баллы (например, при конструировании собственных текстовых признаков). 

Дедлайн: **18 октября 23:59**


### Блок 1: Аугментация с заменой слов на основе ```word2vec``` (2 балла)

* Можно использовать библиотеку ```gensim``` и модели [rusvectores](https://rusvectores.org/ru/models/)



In [1]:
# !wget -c http://vectors.nlpl.eu/repository/20/180.zip
# !unzip 180.zip

In [2]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

model = gensim.models.KeyedVectors.load_word2vec_format("model.bin", binary=True)
model.init_sims(replace=True)

# model_file = 'araneum_none_fasttextcbow_300_5_2018.model'

# model = KeyedVectors.load(model_file)

In [3]:
model["бабушка_NOUN"].shape

(300,)

In [4]:
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import precision_recall_fscore_support
from matplotlib import pyplot
import os
import string
from pymorphy2 import MorphAnalyzer
from razdel import tokenize
from nltk.corpus import stopwords
from tqdm import tqdm

morph = MorphAnalyzer()
stop = set(stopwords.words('english'))

def my_preprocess(text: str):
    text = str(text)
    text = text.replace("\n", " ").replace('/', ' ')
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokenized_text = list(tokenize(text))
    lemm = [morph.parse(i.text)[0].normal_form for i in tokenized_text]
    words = [i for i in lemm if i not in stop]
    return " ".join(words)

Напишите функцию, которая заменяет слова во входном предложении их ближайшими семантическими ассоциатами (1 балл).

In [5]:
def augment_word2vec(sentence, model=model):
    sentence = str(sentence)
    res = []
    for word in sentence.split():
        normal_form = morph.parse(word)[0]
        word_POS = normal_form.normal_form + "_" + str(normal_form.tag.POS)
        if word_POS in model: 
            res.append(model.most_similar(positive=[word_POS], topn=1)[0][0].split("_"))
    res = [i[0] for i in res]
    return " ".join(res)

In [6]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sent = "мама помыла окно"

# пример работы наивной функции
augment_word2vec(sent)

'мама вымывать окошко'

Измените функцию так, чтобы заменялись слова, относящиеся к заданному набору пос-тегов (например, ADJ, NOUN), и сохранялась грамматика (1 балл). 

In [7]:
def augment_word2vec_pos(sentence, model=model):
    res = []
    for word in sentence.split():
        normal_form = morph.parse(word)[0]
        word_POS = normal_form.normal_form + "_" + str(normal_form.tag.POS)
        if word_POS in model:
            for word_closest in model.most_similar(positive=[word_POS]):
                word_closest = word_closest[0].split("_")
                if word_closest[1] == str(normal_form.tag.POS):
                    res.append(word_closest)
                    break
    res = [i[0] for i in res]
    return " ".join(res)

In [8]:
sent = "мама помыла окно"

# пример работы наивной функции
augment_word2vec_pos(sent)

'бабушка вымывать окошко'

### Блок 2: Регрессия; Medium Stories (до 8 баллов)

Датасет можно скачать [здесь](https://yadi.sk/d/90JykTO48fL6qw) или со страницы кегли. Можно взять подвыборку.



Подзадачи:

1.   Эксплоративный анализ данных (1 балл)
2.   Отбор важных признаков, поиск гиперпараметров, минимизация переобучения (2 балла)
3.   Адаптация аугментации данных с заменой слов (1 балл)
4.   Конструирование текстовых признаков (2 балла)
5.   Сравнение качества моделей, выбор наилучшей (1 балл)
6.   Анализ ошибок (1 балл)  

### Эксплоративный анализ данных (1 балл)

In [9]:
import pandas as pd


# можно не использовать колонки с тегами
usecols = ["Title", "Subtitle", "Image", "Author", "Publication", "Year", "Month", "Day", "Reading_Time", "Claps", "url", "Author_url"]

df = pd.read_csv("Medium_Clean.csv", sep=",", usecols=usecols)
df["Claps"] = df["Claps"].astype(int)
data = df.dropna()
data = data.sample(n=10000)
data.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Title,Subtitle,Image,Author,Publication,Year,Month,Day,Reading_Time,Claps,url,Author_url
1278675,The Address for Innovation is 1717,"By Rasheeda N. Creighton, Director, 1717 Innov...",1,Capital One Tech,Capital One Tech,2018,4,16,5,39,https://medium.com/capitalonetech/the-address-...,https://medium.com/@CapitalOneTech
285468,Men are all the Same,Why does he stare at me like that? Ive never s...,1,Willis G. Ford,The Creative Cafe,2018,2,27,2,34,https://thecreative.cafe/men-are-all-the-same-...,https://thecreative.cafe/@whun450
463151,Dance of the Moon,Todays Focail do a Chara,1,Christine Salkin Davis,Poets Unlimited,2017,11,27,1,34,https://medium.com/poets-unlimited/dance-of-th...,https://medium.com/@csdavis2
1340807,Should You Quit Your Day Job?,"Should I quit my day job? Ah, the question so ...",1,Cristofer Jeschke,The Startup,2018,2,13,5,584,https://medium.com/swlh/should-you-quit-your-d...,https://medium.com/@crisjeschke
1348325,You look happy,It is Tuesday,1,Percy Bharucha,The Haven,2018,7,10,1,47,https://medium.com/the-haven/you-look-happy-ee...,https://medium.com/@percy.bharucha


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1278675 to 1295001
Data columns (total 12 columns):
Title           10000 non-null object
Subtitle        10000 non-null object
Image           10000 non-null int64
Author          10000 non-null object
Publication     10000 non-null object
Year            10000 non-null int64
Month           10000 non-null int64
Day             10000 non-null int64
Reading_Time    10000 non-null int64
Claps           10000 non-null int64
url             10000 non-null object
Author_url      10000 non-null object
dtypes: int64(6), object(6)
memory usage: 1015.6+ KB


In [11]:
data.Year.value_counts(normalize=True)

2018    0.5819
2017    0.4181
Name: Year, dtype: float64

In [12]:
data.Month.value_counts(normalize=True)

10    0.0904
3     0.0887
1     0.0885
8     0.0885
11    0.0844
9     0.0842
7     0.0813
5     0.0810
2     0.0809
4     0.0799
6     0.0789
12    0.0733
Name: Month, dtype: float64

In [13]:
print("Среднее количество симвоолов в подзаголовках: ", np.mean([len(i) for i in data.Subtitle.values]))
print("Среднее количество символов в заглавлении: ", np.mean([len(i) for i in data.Title.values]))

Среднее количество симвоолов в подзаголовках:  86.4394
Среднее количество символов в заглавлении:  36.6451


### Отбор важных признаков, поиск гиперпараметров, минимизация переобучения (2 балла)

In [14]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score

In [15]:
data['process_subtitle'] = data.Subtitle.apply(my_preprocess)
data['title_len'] = data['Title'].apply(len)
data['Subtitle_len'] = data['Subtitle'].apply(len)#(lambda x: len(x.split()))
data['tit_sub_len'] = data['Subtitle'].apply(lambda x: len(x.split()))
data['process_subtitle_len'] = data['process_subtitle'].apply(len)#(lambda x: len(x.split()))


In [16]:
data.corr()

,Image,Year,Month,Day,Reading_Time,Claps,title_len,Subtitle_len,tit_sub_len,process_subtitle_len
Image,1.000000,-0.008762,0.013668,0.013434,0.076910,0.041097,-0.300532,-0.542544,-0.514340,-0.544244
Year,-0.008762,1.000000,-0.855630,0.007539,0.034853,0.012627,0.027662,-0.001340,-0.004124,0.001497
Month,0.013668,-0.855630,1.000000,-0.023149,-0.014007,-0.010246,-0.020687,-0.008138,-0.005931,-0.010301
Day,0.013434,0.007539,-0.023149,1.000000,-0.025955,-0.000652,-0.012087,-0.010429,-0.011188,-0.011387
Reading_Time,0.076910,0.034853,-0.014007,-0.025955,1.000000,0.058519,0.100369,-0.049653,-0.050678,-0.048444
Claps,0.041097,0.012627,-0.010246,-0.000652,0.058519,1.000000,0.005435,-0.027065,-0.027720,-0.027579
title_len,-0.300532,0.027662,-0.020687,-0.012087,0.100369,0.005435,1.000000,0.040418,0.022285,0.064245
Subtitle_len,-0.542544,-0.001340,-0.008138,-0.010429,-0.049653,-0.027065,0.040418,1.000000,0.982772,0.968728
tit_sub_len,-0.514340,-0.004124,-0.005931,-0.011188,-0.050678,-0.027720,0.022285,0.982772,1.000000,0.923800
process_subtitle_len,-0.544244,0.001497,-0.010301,-0.011387,-0.048444,-0.027579,0.064245,0.968728,0.923800,1.000000


In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

data[['title_len',
      'Subtitle_len',
      'tit_sub_len',
      'process_subtitle_len']] = scaler.fit_transform(data[['title_len',
                                                            'Subtitle_len',
                                                            'tit_sub_len',
                                                            'process_subtitle_len']])


In [18]:
X = data[['title_len', 'Subtitle_len', "process_subtitle_len", 'tit_sub_len', 'Image', 'Year', 'Reading_Time']].values
y = data.Claps.values

In [19]:
X = data[['Image', 'Year', 'Reading_Time']].values
y = data.Claps.values

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [21]:
models = []

for alpha in [1, 0.1, 0.01, 0.001, 0.0001]:
    for model in [LinearRegression(alpha), Ridge(alpha), Lasso(alpha), ElasticNet(alpha)]:
#         reg = Ridge(alpha=0.01)
        reg = model
        reg.fit(X_train, y_train)
        x_pred = reg.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, x_pred))
        r2 = r2_score(y_test, x_pred)
        models.append((alpha, str(model).split('(')[0], rmse, r2))
        print("Alpha: ", alpha,
              "Model: ", str(model).split('(')[0],
              "RMSE: ", rmse,
              "R2 score: ", r2)

Alpha:  1 Model:  LinearRegression RMSE:  1447.98717311356 R2 score:  0.0030541955519113317
Alpha:  1 Model:  Ridge RMSE:  1447.986941638942 R2 score:  0.0030545142945563875
Alpha:  1 Model:  Lasso RMSE:  1448.0016573679723 R2 score:  0.0030342505010750864
Alpha:  1 Model:  ElasticNet RMSE:  1448.6446032221181 R2 score:  0.002148702752807541
Alpha:  0.1 Model:  LinearRegression RMSE:  1447.98717311356 R2 score:  0.0030541955519113317
Alpha:  0.1 Model:  Ridge RMSE:  1447.9871498834289 R2 score:  0.003054227540014276
Alpha:  0.1 Model:  Lasso RMSE:  1447.9883606234653 R2 score:  0.00305256033921697
Alpha:  0.1 Model:  ElasticNet RMSE:  1447.9893790458766 R2 score:  0.0030511579605789763
Alpha:  0.01 Model:  LinearRegression RMSE:  1447.98717311356 R2 score:  0.0030541955519113317
Alpha:  0.01 Model:  Ridge RMSE:  1447.9871707897196 R2 score:  0.0030541987518606373
Alpha:  0.01 Model:  Lasso RMSE:  1447.9872892524218 R2 score:  0.003054035627627627
Alpha:  0.01 Model:  ElasticNet RMSE:  

In [22]:
print("Best model: ", sorted(models, key=lambda x: x[2])[0])

Best model:  (0.01, 'ElasticNet', 1447.9797679724782, 0.0030643924729785166)


### Конструирование текстовых признаков (2 балла)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(data[['title_len',
                                                          'Subtitle_len',
                                                          "process_subtitle_len",
                                                          'Image',
                                                          'Year',
                                                          'Reading_Time',
                                                          'process_subtitle']],
                                                    data.Claps.values,
                                                    random_state=42)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer(max_features=500)

subtitle_vectors_train = vectorizer.fit_transform(X_train.process_subtitle.values)
X_train = X_train.drop(['process_subtitle'], axis=1)
print(X_train.values.shape)
X_train = np.concatenate((X_train.values, subtitle_vectors_train.toarray()), axis=1)

subtitle_vectors_test = vectorizer.transform(X_test.process_subtitle.values)
X_test = X_test.drop(['process_subtitle'], axis=1)
X_test = np.concatenate((X_test.values, subtitle_vectors_test.toarray()), axis=1)

(7500, 6)


In [25]:
X_train = np.array(X_train).astype(np.float)

In [26]:
models = []

for alpha in [1, 0.1, 0.01, 0.001, 0.0001]:
    for model in [LinearRegression(alpha), Ridge(alpha), Lasso(alpha), ElasticNet(alpha)]:
#         reg = Ridge(alpha=0.01)
        reg = model
        reg.fit(X_train, y_train)
        x_pred = reg.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, x_pred))
        r2 = r2_score(y_test, x_pred)
        models.append((alpha, str(model).split('(')[0], rmse, r2))
        print("Alpha: ", alpha,
              "Model: ", str(model).split('(')[0],
              "RMSE: ", rmse,
              "R2 score: ", r2)

Alpha:  1 Model:  LinearRegression RMSE:  1467.2551437222348 R2 score:  -0.023654504767798956
Alpha:  1 Model:  Ridge RMSE:  1462.9834329773785 R2 score:  -0.01770272381387894
Alpha:  1 Model:  Lasso RMSE:  1458.387368727978 R2 score:  -0.01131840015194907
Alpha:  1 Model:  ElasticNet RMSE:  1448.426774979116 R2 score:  0.0024487678653692946
Alpha:  0.1 Model:  LinearRegression RMSE:  1467.2551437222348 R2 score:  -0.023654504767798956
Alpha:  0.1 Model:  Ridge RMSE:  1466.7489741398144 R2 score:  -0.0229483516468314
Alpha:  0.1 Model:  Lasso RMSE:  1462.762050762578 R2 score:  -0.017394744610069734
Alpha:  0.1 Model:  ElasticNet RMSE:  1447.677011076612 R2 score:  0.0034812457771293825
Alpha:  0.01 Model:  LinearRegression RMSE:  1467.2551437222348 R2 score:  -0.023654504767798956
Alpha:  0.01 Model:  Ridge RMSE:  1467.2035762778046 R2 score:  -0.023582552293277326
Alpha:  0.01 Model:  Lasso RMSE:  1466.7052543346063 R2 score:  -0.022887369928864087
Alpha:  0.01 Model:  ElasticNet RMS

In [27]:
print("Best model: ", sorted(models, key=lambda x: x[2])[0])

Best model:  (0.01, 'ElasticNet', 1447.139593879671, 0.0042209783373564624)


### Адаптация аугментации данных с заменой слов (1 балл)

In [28]:
data2 = data[[i for i in data.columns if i != 'Title' and i != 'Subtitle']]

In [29]:
data2.head()

,Image,Author,Publication,Year,Month,Day,Reading_Time,Claps,url,Author_url,process_subtitle,title_len,Subtitle_len,tit_sub_len,process_subtitle_len
1278675,1,Capital One Tech,Capital One Tech,2018,4,16,5,39,https://medium.com/capitalonetech/the-address-...,https://medium.com/@CapitalOneTech,rasheeda n creighton director 1717 innovation ...,-0.135616,-0.226782,-0.416568,0.054884
285468,1,Willis G. Ford,The Creative Cafe,2018,2,27,2,34,https://thecreative.cafe/men-are-all-the-same-...,https://thecreative.cafe/@whun450,stare like ive never seen dude looks like problem,-0.853407,0.434202,0.837652,-0.255135
463151,1,Christine Salkin Davis,Poets Unlimited,2017,11,27,1,34,https://medium.com/poets-unlimited/dance-of-th...,https://medium.com/@csdavis2,todays focail chara,-1.007219,-0.917143,-0.834641,-0.875172
1340807,1,Cristofer Jeschke,The Startup,2018,2,13,5,584,https://medium.com/swlh/should-you-quit-your-d...,https://medium.com/@crisjeschke,quit day job ah question many us ask,-0.391970,-0.373667,-0.082110,-0.523818
1348325,1,Percy Bharucha,The Haven,2018,7,10,1,47,https://medium.com/the-haven/you-look-happy-ee...,https://medium.com/@percy.bharucha,tuesday,-1.161032,-1.078716,-1.001871,-1.123187


In [30]:
from tqdm import tqdm

title_buf = []
subtitle_buf = []

for i in tqdm(data.Title.values):
    title_buf.append(augment_word2vec(i))

for i in tqdm(data.Subtitle.values):
    subtitle_buf.append(augment_word2vec(i))

data2["Title"] = title_buf #data.Title.apply(augment_word2vec)
data2["Subtitle"] = subtitle_buf #data.Subtitle.apply(augment_word2vec)

100%|██████████| 10000/10000 [00:07<00:00, 1324.30it/s]
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [31]:
data_augm = pd.concat([data, data2], ignore_index=True)

In [32]:
data_augm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
Author                  20000 non-null object
Author_url              20000 non-null object
Claps                   20000 non-null int64
Day                     20000 non-null int64
Image                   20000 non-null int64
Month                   20000 non-null int64
Publication             20000 non-null object
Reading_Time            20000 non-null int64
Subtitle                20000 non-null object
Subtitle_len            20000 non-null float64
Title                   20000 non-null object
Year                    20000 non-null int64
process_subtitle        20000 non-null object
process_subtitle_len    20000 non-null float64
tit_sub_len             20000 non-null float64
title_len               20000 non-null float64
url                     20000 non-null object
dtypes: float64(4), int64(6), object(7)
memory usage: 2.6+ MB


In [33]:
len(title_buf)

10000

In [34]:
data_augm['title_len'] = data_augm['Title'].apply(len)
data_augm['Subtitle_len'] = data_augm['Subtitle'].apply(len)#(lambda x: len(x.split()))

In [35]:
data_augm['process_subtitle'] = data_augm.Subtitle.apply(my_preprocess)
data_augm['title_len'] = data_augm['Title'].apply(len)
data_augm['Subtitle_len'] = data_augm['Subtitle'].apply(len)#(lambda x: len(x.split()))
data_augm['tit_sub_len'] = data_augm['Subtitle'].apply(lambda x: len(x.split()))
data_augm['process_subtitle_len'] = data_augm['process_subtitle'].apply(len)#(lambda x: len(x.split()))

In [36]:
X_train, X_test, y_train, y_test = train_test_split(data_augm[['title_len',
                                                          'Subtitle_len',
                                                          "process_subtitle_len",
                                                          'Image',
                                                          'Year',
                                                          'Reading_Time',
                                                          'process_subtitle']],
                                                    data_augm.Claps.values,
                                                    random_state=42)

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer(max_features=500)

subtitle_vectors_train = vectorizer.fit_transform(X_train.process_subtitle.values)
X_train = X_train.drop(['process_subtitle'], axis=1)
print(X_train.values.shape)
X_train = np.concatenate((X_train.values, subtitle_vectors_train.toarray()), axis=1)

subtitle_vectors_test = vectorizer.transform(X_test.process_subtitle.values)
X_test = X_test.drop(['process_subtitle'], axis=1)
X_test = np.concatenate((X_test.values, subtitle_vectors_test.toarray()), axis=1)

(15000, 6)


In [38]:
models = []

for alpha in [1, 0.1, 0.01, 0.001, 0.0001]:
    for model in [LinearRegression(alpha), Ridge(alpha), Lasso(alpha), ElasticNet(alpha)]:
#         reg = Ridge(alpha=0.01)
        reg = model
        reg.fit(X_train, y_train)
        x_pred = reg.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, x_pred))
        r2 = r2_score(y_test, x_pred)
        models.append((alpha, str(model).split('(')[0], rmse, r2))
        print("Alpha: ", alpha,
              "Model: ", str(model).split('(')[0],
              "RMSE: ", rmse,
              "R2 score: ", r2)

Alpha:  1 Model:  LinearRegression RMSE:  2523.7548717308823 R2 score:  -0.004040489573794215
Alpha:  1 Model:  Ridge RMSE:  2521.3909516366675 R2 score:  -0.002160465536560574
Alpha:  1 Model:  Lasso RMSE:  2515.8461618583674 R2 score:  0.0022423892905519827
Alpha:  1 Model:  ElasticNet RMSE:  2513.36869026838 R2 score:  0.004206499072673675
Alpha:  0.1 Model:  LinearRegression RMSE:  2523.7548717308823 R2 score:  -0.004040489573794215
Alpha:  0.1 Model:  Ridge RMSE:  2523.4723679638946 R2 score:  -0.0038157218292182993
Alpha:  0.1 Model:  Lasso RMSE:  2521.296307439298 R2 score:  -0.0020852317512636276
Alpha:  0.1 Model:  ElasticNet RMSE:  2512.595965123801 R2 score:  0.004818710400043891
Alpha:  0.01 Model:  LinearRegression RMSE:  2523.7548717308823 R2 score:  -0.004040489573794215
Alpha:  0.01 Model:  Ridge RMSE:  2523.7260575060272 R2 score:  -0.004017563033977822
Alpha:  0.01 Model:  Lasso RMSE:  2523.385215574879 R2 score:  -0.0037463860768800483
Alpha:  0.01 Model:  ElasticNet

### Сравнение качества моделей, выбор наилучшей (1 балл)

Модель с дополнительными аугментированными данными показала результат лучше исходных данных, поэтому возьмём наилучшую модель на аугментированных данных

In [39]:
print("Best model: ", sorted(models, key=lambda x: x[2])[0])

Best model:  (0.01, 'ElasticNet', 2512.492309624437, 0.00490081980971091)


### Анализ ошибок (1 балл)

Слишком низкий показатель r2 и слишком большая ошибка, пыталась сделать всё по чёткому плану, ноrmse остаётся слишком большим при любых изменениях. Можно выделить другие признаки, не только вектора tf-idf и обучить на них нелинейные модели, для получения более глубоких зависимостей.

### Блок 3: Paraphrase Detection (до 8 баллов)


* [Статья по русскоязычному корпусу парафразов](https://www.aclweb.org/anthology/2020.ngt-1.6/)
* [Еще статья](http://www.dialog-21.ru/media/3928/loukachevitchnvetal.pdf)
* [Сайт корпуса](http://paraphraser.ru/)

Отформатированные данные можно скачать [здесь](https://yadi.sk/d/OIgbnRA6OVJ4VQ)


1.   Эксплоративный анализ данных (1 балл)
2.   Поиск гиперпараметров, минимизация переобучения (2 балла)
3.   Аугментация данных (1 балл)
4.   Сравнение качества моделей, выбор наилучшей (1 балл)
5.   Конструирование текстовых признаков (2 балла)
6.   Анализ ошибок (1 балл)



In [40]:
para = pd.read_csv("paraphrases.tsv", sep="\t")
para.head()

,text_1,text_2,class
0,Совет юстиции Бразилии легализовал однополые б...,Совет юстиции Бразилии разрешил однополые браки.,1
1,"""Магнит"" поручится перед ""Абсолют Банком"" за к...",Выпуск сигарет в России упал из-за антитабачно...,-1
2,ЕС призвал США не бомбить Сирию до публикации ...,Евросоюз призвал США дождаться доклада ООН по ...,-1
3,Депо Московского метрополитена впервые перейде...,Частной компании впервые отдадут депо в москов...,1
4,Два человека погибли в столкновениях между кур...,Один человек погиб при столкновении судов у бе...,-1


### Эксплоративный анализ данных (1 балл)

##### В датасете много повторений, оставим только уникальные текстовые пары

In [41]:
len(set(para.text_1))

3789

In [42]:
non_duplicate = []
normal = []
for value in para.values:
    if (value[0] + value[1] not in non_duplicate):
        normal.append(value)
    non_duplicate.append(value[0] + value[1])
    non_duplicate.append(value[1] + value[0])
len(normal)

4256

In [43]:
data = pd.DataFrame(normal, columns=['text_1', 'text_2', 'class'])
data.head()

,text_1,text_2,class
0,Совет юстиции Бразилии легализовал однополые б...,Совет юстиции Бразилии разрешил однополые браки.,1
1,"""Магнит"" поручится перед ""Абсолют Банком"" за к...",Выпуск сигарет в России упал из-за антитабачно...,-1
2,ЕС призвал США не бомбить Сирию до публикации ...,Евросоюз призвал США дождаться доклада ООН по ...,-1
3,Депо Московского метрополитена впервые перейде...,Частной компании впервые отдадут депо в москов...,1
4,Два человека погибли в столкновениях между кур...,Один человек погиб при столкновении судов у бе...,-1


In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4256 entries, 0 to 4255
Data columns (total 3 columns):
text_1    4256 non-null object
text_2    4256 non-null object
class     4256 non-null int64
dtypes: int64(1), object(2)
memory usage: 99.8+ KB


In [45]:
print("Среднее количество символов в text_1: ", np.mean([len(i) for i in para.text_1.values]))
print("Среднее количество символов в text_2: ", np.mean([len(i) for i in para.text_2.values]))

Среднее количество символов в text_1:  58.5288056206089
Среднее количество символов в text_2:  55.04074941451991


### Конструирование текстовых признаков (2 балла)

In [46]:
data['text_1_normal'] = data.text_1.apply(my_preprocess)
data['text_2_normal'] = data.text_2.apply(my_preprocess)
data.head()

,text_1,text_2,class,text_1_normal,text_2_normal
0,Совет юстиции Бразилии легализовал однополые б...,Совет юстиции Бразилии разрешил однополые браки.,1,совет юстиция бразилия легализовать однополый ...,совет юстиция бразилия разрешить однополый брак
1,"""Магнит"" поручится перед ""Абсолют Банком"" за к...",Выпуск сигарет в России упал из-за антитабачно...,-1,магнит поручиться перед абсолют банк за кредит...,выпуск сигарета в россия упасть изз антитабачн...
2,ЕС призвал США не бомбить Сирию до публикации ...,Евросоюз призвал США дождаться доклада ООН по ...,-1,ес призвать сша не бомбить сирия до публикация...,евросоюз призвать сша дождаться доклад оон по ...
3,Депо Московского метрополитена впервые перейде...,Частной компании впервые отдадут депо в москов...,1,депо московский метрополитен впервые перейти в...,частный компания впервые отдать депо в московс...
4,Два человека погибли в столкновениях между кур...,Один человек погиб при столкновении судов у бе...,-1,два человек погибнуть в столкновение между кур...,один человек погибнуть при столкновение судно ...


In [47]:
X_train, X_test, y_train, y_test = train_test_split(data[['text_1_normal','text_2_normal']],
                                                    data['class'].values,
                                                    random_state=42)

In [48]:
vectorizer = TfidfVectorizer(max_features=5000)
vectorizer.fit(set(X_train.text_1_normal.values + X_train.text_2_normal.values))


vec_1 = vectorizer.transform(X_train.text_1_normal.values)
vec_2 = vectorizer.transform(X_train.text_2_normal.values)

In [50]:
from scipy.spatial import distance

# cos_sim = np.linalg.norm(sub, axis=1)
# vec_1 = normalize(vec_1, axis=1, norm='l2')
# vec_2 = normalize(vec_2, axis=1, norm='l2')

cos_sim = []
for i in range(vec_1.shape[0]):
    if np.linalg.norm(vec_1[i, :].toarray()) == 0 or np.linalg.norm(vec_2[i, :].toarray()) == 0:
        cos_sim.append(1.0)
        continue

    cos_sim.append( distance.cosine(vec_1[i, :].toarray(), vec_2[i, :].toarray()))
    
# cos_sim = vec_1.dot(vec_2.T).sum(axis=1)
X_train['cos_sim'] = np.asarray(cos_sim).reshape(-1)
X_train = X_train[['cos_sim']].values

In [51]:
vec_1 = vectorizer.transform(X_test.text_1_normal.values)
vec_2 = vectorizer.transform(X_test.text_2_normal.values)

# sub = vec_1 - vec_2

# vec_1 = normalize(vec_1, axis=1, norm='l2')
# vec_2 = normalize(vec_2, axis=1, norm='l2')

# cos_sim = vec_1.dot(vec_2.T).sum(axis=1)
# cos_sim = normalize(vec_1 - vec_2, axis=1, norm='l2').sum(axis=1)
cos_sim = []
for i in range(vec_1.shape[0]):
    if np.linalg.norm(vec_1[i, :].toarray()) == 0 or np.linalg.norm(vec_2[i, :].toarray()) == 0:
        cos_sim.append(1.0)
        continue

    cos_sim.append( distance.cosine(vec_1[i, :].toarray(), vec_2[i, :].toarray()))

X_test['cos_sim'] = np.asarray(cos_sim).reshape(-1)
X_test = X_test[['cos_sim']].values

### Поиск гиперпараметров, минимизация переобучения (2 балла)

In [52]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


model = LogisticRegression()
reg = model
reg.fit(X_train, y_train)
x_pred = reg.predict(X_test)
acc = accuracy_score(y_test, x_pred.round())
r2 = r2_score(y_test, x_pred)
models.append((alpha, str(model).split('(')[0], rmse, r2))
print("Model: ", str(model).split('(')[0],
      "accuracy: ", acc)

Model:  LogisticRegression accuracy:  0.8045112781954887


### Аугментация данных (1 балл)

In [53]:
data2 = pd.DataFrame()

In [54]:
data['text_1_normal'] = data.text_1_normal.values

In [55]:
buf = []

for i in tqdm(data.text_1_normal.values):
    buf.append(augment_word2vec(i))

data2['text_1_normal'] = buf
buf = []

for i in tqdm(data.text_2_normal.values):
    buf.append(augment_word2vec(i))
    
data2['text_2_normal'] = buf
data2['class'] = data['class']

100%|██████████| 4256/4256 [03:12<00:00, 22.05it/s]


In [56]:
data = pd.concat([data[['text_1_normal', 'text_2_normal', 'class']],
                  data2[['text_1_normal', 'text_2_normal', 'class']] ])

### Сравнение качества моделей, выбор наилучшей (1 балл)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(data[['text_1_normal','text_2_normal']],
                                                    data['class'].values,
                                                    random_state=3)

In [60]:
vectorizer = TfidfVectorizer(max_features=5000)
vectorizer.fit(set(X_train.text_1_normal.values + X_train.text_2_normal.values))


vec_1 = vectorizer.transform(X_train.text_1_normal.values)
vec_2 = vectorizer.transform(X_train.text_2_normal.values)

In [61]:
#  cos_sim = np.linalg.norm(sub, axis=1)
# vec_1 = normalize(vec_1, axis=1, norm='l2')
# vec_2 = normalize(vec_2, axis=1, norm='l2')

cos_sim = []
for i in range(vec_1.shape[0]):
    if np.linalg.norm(vec_1[i, :].toarray()) == 0 or np.linalg.norm(vec_2[i, :].toarray()) == 0:
        cos_sim.append(1.0)
        continue

    cos_sim.append( distance.cosine(vec_1[i, :].toarray(), vec_2[i, :].toarray()))
    
# cos_sim = vec_1.dot(vec_2.T).sum(axis=1)
X_train['cos_sim'] = np.asarray(cos_sim).reshape(-1)
X_train = X_train[['cos_sim']].values

In [62]:
vec_1 = vectorizer.transform(X_test.text_1_normal.values)
vec_2 = vectorizer.transform(X_test.text_2_normal.values)

# sub = vec_1 - vec_2

# vec_1 = normalize(vec_1, axis=1, norm='l2')
# vec_2 = normalize(vec_2, axis=1, norm='l2')

# cos_sim = vec_1.dot(vec_2.T).sum(axis=1)
# cos_sim = normalize(vec_1 - vec_2, axis=1, norm='l2').sum(axis=1)
cos_sim = []
for i in range(vec_1.shape[0]):
    if np.linalg.norm(vec_1[i, :].toarray()) == 0 or np.linalg.norm(vec_2[i, :].toarray()) == 0:
        cos_sim.append(1.0)
        continue

    cos_sim.append( distance.cosine(vec_1[i, :].toarray(), vec_2[i, :].toarray()))

X_test['cos_sim'] = np.asarray(cos_sim).reshape(-1)
X_test = X_test[['cos_sim']].values

In [63]:
model = LogisticRegression()
reg = model
reg.fit(X_train, y_train)
x_pred = reg.predict(X_test)
acc = accuracy_score(y_test, x_pred.round())
r2 = r2_score(y_test, x_pred)
models.append((alpha, str(model).split('(')[0], rmse, r2))
print("Model: ", str(model).split('(')[0],
      "accuracy: ", acc)

Model:  LogisticRegression accuracy:  0.7720864661654135


С аугментированными данными accuracy модели ниже, чем без них

### Анализ ошибок (1 балл)

Наверно для улучшения качества модели надо было дополнительно нормализовать значения и вектора, чтобы более обусловленно подходить к расстоянию между документами. Также можно было бы взять другие векторизаторы и сконкатенировать несколько эмбеддингов для улучшения качества и испольовать другие показатели схожести в модели, а не только расстояние между векторами.